# Using sqlalchemy and pandas to read and write from and to a local sqlite database for Fiscal Data

This post includes code adapted from [python for finance and trading algorithms udemy course](https://udemy.com/python-for-finance-and-trading-algorithms/) and [python for finance and trading algorithms udemy course notebooks](https://github.com/theoneandonlywoj/Python-for-Financial-Analysis-and-Algorithmic-Trading) and the [documentation here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.to_sql.html)

In [28]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
df=pd.read_csv('df_panel_fix.csv')

In [30]:
df

,Unnamed: 0,province,specific,general,year,gdp,fdi,rnr,rr,i,fr,reg,it
0,0,Anhui,147002.0,NaN,1996,2093.30,50661,0.000000,0.000000,0.000000,1128873,East China,631930
1,1,Anhui,151981.0,NaN,1997,2347.32,43443,0.000000,0.000000,0.000000,1356287,East China,657860
2,2,Anhui,174930.0,NaN,1998,2542.96,27673,0.000000,0.000000,0.000000,1518236,East China,889463
3,3,Anhui,285324.0,NaN,1999,2712.34,26131,NaN,NaN,NaN,1646891,East China,1227364
4,4,Anhui,195580.0,32100.0,2000,2902.09,31847,0.000000,0.000000,0.000000,1601508,East China,1499110
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,355,Zhejiang,391292.0,260313.0,2003,9705.02,498055,1.214286,0.035714,0.035714,6217715,East China,2261631
356,356,Zhejiang,656175.0,276652.0,2004,11648.70,668128,1.214286,0.035714,0.035714,NaN,East China,3162299
357,357,Zhejiang,656175.0,NaN,2005,13417.68,772000,1.214286,0.035714,0.035714,NaN,East China,2370200
358,358,Zhejiang,1017303.0,394795.0,2006,15718.47,888935,1.214286,0.035714,0.035714,11537149,East China,2553268


In [31]:
df_subset = df[["year", "reg", "province", "it", "specific", 'gdp',"fdi"]]
df_subset

,year,reg,province,it,specific,gdp,fdi
0,1996,East China,Anhui,631930,147002.0,2093.30,50661
1,1997,East China,Anhui,657860,151981.0,2347.32,43443
2,1998,East China,Anhui,889463,174930.0,2542.96,27673
3,1999,East China,Anhui,1227364,285324.0,2712.34,26131
4,2000,East China,Anhui,1499110,195580.0,2902.09,31847
...,...,...,...,...,...,...,...
355,2003,East China,Zhejiang,2261631,391292.0,9705.02,498055
356,2004,East China,Zhejiang,3162299,656175.0,11648.70,668128
357,2005,East China,Zhejiang,2370200,656175.0,13417.68,772000
358,2006,East China,Zhejiang,2553268,1017303.0,15718.47,888935


In [32]:
df_subset.columns = ["year", "region", "province", "it", "specific", 'gdp',"fdi"]

In [33]:
df_subset

,year,region,province,it,specific,gdp,fdi
0,1996,East China,Anhui,631930,147002.0,2093.30,50661
1,1997,East China,Anhui,657860,151981.0,2347.32,43443
2,1998,East China,Anhui,889463,174930.0,2542.96,27673
3,1999,East China,Anhui,1227364,285324.0,2712.34,26131
4,2000,East China,Anhui,1499110,195580.0,2902.09,31847
...,...,...,...,...,...,...,...
355,2003,East China,Zhejiang,2261631,391292.0,9705.02,498055
356,2004,East China,Zhejiang,3162299,656175.0,11648.70,668128
357,2005,East China,Zhejiang,2370200,656175.0,13417.68,772000
358,2006,East China,Zhejiang,2553268,1017303.0,15718.47,888935


In [34]:
import sqlalchemy as db
from sqlalchemy import create_engine
import sqlite3
import pandas as pd

In [35]:
engine = db.create_engine('sqlite:///fiscal.db')
connection = engine.connect()
metadata = db.MetaData()

In [36]:
fiscal_table = db.Table('fiscal_table', metadata, 
    db.Column('year',db.Integer, nullable=True, index=False),
    db.Column('region',db.Integer, nullable=True),
    db.Column('province',db.Integer, nullable=True),
    db.Column('it',db.Integer, nullable=True),
    db.Column('specific',db.Integer, nullable=True),
    db.Column('gdp',db.Integer, nullable=True),
    db.Column('fdi', db.Numeric, nullable=True)
)

In [37]:
metadata.create_all(engine) #Creates the table

In [38]:
df_subset.to_sql('fiscal_table', con=engine, if_exists='append', index=False)

In [39]:
engine.execute("SELECT * FROM fiscal_table LIMIT 10").fetchall()

[(1996, 'East China', 'Anhui', 631930, 147002, 2093.3, 50661),
 (1997, 'East China', 'Anhui', 657860, 151981, 2347.32, 43443),
 (1998, 'East China', 'Anhui', 889463, 174930, 2542.96, 27673),
 (1999, 'East China', 'Anhui', 1227364, 285324, 2712.34, 26131),
 (2000, 'East China', 'Anhui', 1499110, 195580, 2902.09, 31847),
 (2001, 'East China', 'Anhui', 2165189, 250898, 3246.71, 33672),
 (2002, 'East China', 'Anhui', 2404936, 434149, 3519.72, 38375),
 (2003, 'East China', 'Anhui', 2815820, 619201, 3923.11, 36720),
 (2004, 'East China', 'Anhui', 3422176, 898441, 4759.3, 54669),
 (2005, 'East China', 'Anhui', 3874846, 898441, 5350.17, 69000)]

In [40]:
sql = """
SELECT
  year
, region
, province
, it
--, CURRENT_DATE()
FROM fiscal_table
"""

cnxn = connection

In [41]:
df = pd.read_sql(sql, cnxn)

In [42]:
df.tail(30)

,year,region,province,it
330,2002,Northwest China,Xinjiang,2150325
331,2003,Northwest China,Xinjiang,2355164
332,2004,Northwest China,Xinjiang,2838346
333,2005,Northwest China,Xinjiang,3421743
334,2006,Northwest China,Xinjiang,4686125
335,2007,Northwest China,Xinjiang,5502470
336,1996,Southwest China,Yunnan,1374111
337,1997,Southwest China,Yunnan,1452425
338,1998,Southwest China,Yunnan,1617463
339,1999,Southwest China,Yunnan,1888666


In [78]:
#df['it'].plot(figsize = (12, 8))